# Notes

## 2022-01-03 ##
Here I'm attempting figure out why my rust implementation all of sudden isn't generating the same number of pathways as the cdp python implementation
(sometimes it's more, sometimes less?!).

In [10]:
import pandas as pd
import holoviews as hv
import hvplot.pandas
import json
from collections import defaultdict, Counter
from itertools import chain
from pprint import pprint

In [11]:
case = 'sq' # 
assert case in ['ts', 'sq']
f_py = "/Users/oste814/projects/pacrat/code/cdp/examples/paths_{}.json".format(case) #sqpaths.json"
f_rs = "/Users/oste814/projects/pacrat/code/pacrat-backend/algo/paths_{}.json".format(case) #examples/small/paths.json"

In [12]:
with open(f_py) as f:
    j_py = json.load(f)
len(j_py)

4487

In [13]:
with open(f_rs) as f:
    j_rs = json.load(f)
len(j_rs)

100000

In [14]:
j_rs[0]

{'id': '6e494a1f-be78-47c6-87a4-23b9b6c7d3f9',
 'scenario_id': 'd9dc3302-93e9-4549-b360-dbc4d0693755',
 'critical_detection_point': {'type': 'Edge',
  'id': '4980bf1f-b50b-432a-ab35-8949b00aa37d'},
 'interdiction_probability': 0.14454082,
 'steps': [{'type': 'Area', 'id': '25d1c743-9a91-473e-9f3b-f1eb011ab252'},
  {'type': 'Safeguard', 'id': '385c51fe-d4cd-4130-a2fa-89294932ead2'},
  {'type': 'Safeguard', 'id': '7948c789-858b-45fa-a1cb-74b66721ada4'},
  {'type': 'Area', 'id': '45898c69-28c0-4c5c-9fde-24676114650d'},
  {'type': 'Safeguard', 'id': 'd187ad37-ab53-4e94-8704-42961b52e024'},
  {'type': 'Safeguard', 'id': '88d1c374-c8bd-4e42-9b67-e5b4c851aa83'},
  {'type': 'Area', 'id': 'b16c142b-3c0c-44bc-8250-e7fb23d3648c'},
  {'type': 'Safeguard', 'id': '1981964b-7f3a-4de8-abb3-f212ea87dc11'},
  {'type': 'Area', 'id': '1e240fcd-b8b5-4afe-a0b8-3617ce0c8e28'},
  {'type': 'Safeguard', 'id': '61225ca1-2c84-4534-8a2b-6269b3fa824a'},
  {'type': 'Area', 'id': '8b148d7f-1f2e-4e74-8a8c-e9e7a883f184

In [15]:
def get_node_to_paths_map_alt(paths_json):
    _map = defaultdict(lambda: defaultdict(list))#defaultdict(list)
    for path in paths_json:
        sub_map = {}
        key = list()
        sgs = list()
        acs = list()
        val = list()
        area_id = None
        for step in path['steps']:
            _id = step['id']
            _type = step['type']
            if _type == 'Area':
                if area_id is not None:
                    val.append((area_id, tuple(sorted(acs)), tuple(sorted(sgs))))
                sgs.clear()
                acs.clear()
                area_id = _id
                key.append(area_id)
            elif _type == 'Action':
                acs.append(_id)
            elif _type == 'Safeguard':
                sgs.append(_id)
            else:
                raise Exception("oops, _type {} not supported".format(_type))
        if area_id is not None: # and (len(acs) + len(sgs) > 0 or area_id == _id):
            val.append((area_id, tuple(sorted(acs)), tuple(sorted(sgs))))
        sgs.clear()
        acs.clear()
        # _map[tuple(key)].append((tuple(val), path['interdiction_probability'], path['critical_detection_point']['id'], path['critical_detection_point']['type']))
        # _map[tuple(key)][tuple(val)].append((path['interdiction_probability'], path['critical_detection_point']['id'], path['critical_detection_point']['type']))
        _map[tuple(key)][tuple(val)].append((path['interdiction_probability'], path['critical_detection_point']['id']))#, path['critical_detection_point']['type']))
    # ret = {}
    # for k, v in _map.items():
    #     s = set(v)
    #     if len(s) != len(v):
    #         print("oops, duplicate must be at v of k={}, since len(v) {} > len(set(v)) {}".format(k, len(v), len(s)))
    #     ret[k] = s
    return _map #ret #{k: set(v) for k, v in _map.items()}

In [16]:
def get_node_to_paths_map(paths_json):
    _map = defaultdict(lambda: defaultdict(list))#defaultdict(list)
    for path in paths_json:
        sub_map = {}
        key = list()
        sgs = list()
        acs = list()
        val = list()
        area_id = None
        for step in path['steps']:
            _id = step['id']
            _type = step['type']
            if _type == 'Area':
                if area_id is not None:
                    val.append((area_id, tuple(sorted(acs)), tuple(sorted(sgs))))
                sgs.clear()
                acs.clear()
                area_id = _id
                key.append(area_id)
            elif _type == 'Action':
                acs.append(_id)
            elif _type == 'Safeguard':
                sgs.append(_id)
            else:
                raise Exception("oops, _type {} not supported".format(_type))
        if area_id is not None: # and (len(acs) + len(sgs) > 0 or area_id == _id):
            val.append((area_id, tuple(sorted(acs)), tuple(sorted(sgs))))
        sgs.clear()
        acs.clear()
        # _map[tuple(key)].append((tuple(val), path['interdiction_probability'], path['critical_detection_point']['id'], path['critical_detection_point']['type']))
        _map[tuple(key)][tuple(val)].append((path['interdiction_probability'], path['critical_detection_point']['id'], path['critical_detection_point']['type']))
    # ret = {}
    # for k, v in _map.items():
    #     s = set(v)
    #     if len(s) != len(v):
    #         print("oops, duplicate must be at v of k={}, since len(v) {} > len(set(v)) {}".format(k, len(v), len(s)))
    #     ret[k] = s
    return _map #ret #{k: set(v) for k, v in _map.items()}

In [17]:
j_rs[0]['steps']

[{'type': 'Area', 'id': '25d1c743-9a91-473e-9f3b-f1eb011ab252'},
 {'type': 'Safeguard', 'id': '385c51fe-d4cd-4130-a2fa-89294932ead2'},
 {'type': 'Safeguard', 'id': '7948c789-858b-45fa-a1cb-74b66721ada4'},
 {'type': 'Area', 'id': '45898c69-28c0-4c5c-9fde-24676114650d'},
 {'type': 'Safeguard', 'id': 'd187ad37-ab53-4e94-8704-42961b52e024'},
 {'type': 'Safeguard', 'id': '88d1c374-c8bd-4e42-9b67-e5b4c851aa83'},
 {'type': 'Area', 'id': 'b16c142b-3c0c-44bc-8250-e7fb23d3648c'},
 {'type': 'Safeguard', 'id': '1981964b-7f3a-4de8-abb3-f212ea87dc11'},
 {'type': 'Area', 'id': '1e240fcd-b8b5-4afe-a0b8-3617ce0c8e28'},
 {'type': 'Safeguard', 'id': '61225ca1-2c84-4534-8a2b-6269b3fa824a'},
 {'type': 'Area', 'id': '8b148d7f-1f2e-4e74-8a8c-e9e7a883f184'},
 {'type': 'Action', 'id': 'ab0d0281-adfb-4f4c-a7b0-c0c37a223ec7'},
 {'type': 'Action', 'id': '170161eb-c664-4bad-aa7f-69fbe6f7597b'},
 {'type': 'Safeguard', 'id': '69fe57be-ea39-4e59-97ea-24a83602ed5f'},
 {'type': 'Area', 'id': 'eb827a91-123a-45bd-bc75-b7

In [18]:
# sorted(rs_map[
#     ('5aac57dd-62e2-40ce-b033-46044f4a571a', '18fb6481-3fbc-42d9-8f57-1a17fcbfbae5', '51eb9a20-1c6d-4ae5-a1d2-ae07e199a327', 'ce52e812-98c7-4d87-b1f3-807d583463ca', 'bfe3bea9-11e7-4408-8127-039627c5dd6b', 'dc633730-cd2a-4449-ac6c-fa92cf01640f')
#       ])[0]

In [19]:
rs_map = get_node_to_paths_map_alt(j_rs)
len(rs_map)

7243

In [20]:
py_map = get_node_to_paths_map(j_py)
len(py_map)

105

In [21]:
set(rs_map) == set(py_map)

False

In [22]:
print('rs key lens', sorted(Counter([len(k) for k in rs_map]).items()))
print('py key lens', sorted(Counter([len(k) for k in py_map]).items()))

rs key lens [(8, 6), (9, 12), (10, 48), (11, 83), (12, 183), (13, 213), (14, 375), (15, 443), (16, 675), (17, 769), (18, 991), (19, 1065), (20, 812), (21, 518), (22, 357), (23, 300), (24, 206), (25, 163), (26, 24)]
py key lens [(4, 1), (5, 1), (7, 2), (8, 16), (9, 24), (10, 61)]


In [58]:
opts = {
    'width': 1000,
    'height': 400,
}

In [59]:
df = pd.DataFrame([[len(k), len(v)] for k, v in rs_map.items()], 
                  columns=['path_len', 'overlay_count'])

_df = df.groupby('path_len')[['overlay_count']].count().rename(columns={'overlay_count': 'path_count'})
p1 = (_df.hvplot.line(title='Number of pathways of each length, for top {} scores'.format(len(j_rs)), **opts) * \
      _df.hvplot.scatter(**opts))
_df = df.groupby('path_len')[['overlay_count']].sum()
p2 = (_df.hvplot.line(title='Number of pathway overlays of each pathway length, for top {} scores'.format(len(j_rs)), **opts) * \
      _df.hvplot.scatter(**opts))
(p1 + p2).cols(1)

:Layout
   .Overlay.I  :Overlay
      .Curve.I   :Curve   [path_len]   (path_count)
      .Scatter.I :Scatter   [path_len]   (path_count)
   .Overlay.II :Overlay
      .Curve.I   :Curve   [path_len]   (overlay_count)
      .Scatter.I :Scatter   [path_len]   (overlay_count)

In [60]:
df.sort_values('path_len').hvplot.box(y='overlay_count', by='path_len', title='Distribution of number of overlays by path length', **opts)

:BoxWhisker   [path_len]   (overlay_count)

In [89]:
df = pd.DataFrame([[len(k), len(subv), min(tuple(zip(*subv))[0]), max(tuple(zip(*subv))[0]), min(tuple(zip(*subv))[1]), max(tuple(zip(*subv))[1])] for k, v in rs_map.items() for subk, subv in v.items()], 
                  columns=['path_len', 'duplicate_count', 'min_ip', 'max_ip', 'min_cdp', 'max_cdp'])

if (df['min_ip'] == df['max_ip']).all():
    df['ip'] = df['min_ip']
    df['cdp'] = df['min_cdp']
else:
    print('oops, find what to set ip/cdp to now that duplicates exist!?')
df.describe()

,path_len,duplicate_count,min_ip,max_ip,ip
count,100000.000000,100000.0,100000.000000,100000.000000,100000.000000
mean,18.652490,1.0,0.134257,0.134257,0.134257
std,3.296531,0.0,0.013419,0.013419,0.013419
min,8.000000,1.0,0.053369,0.053369,0.053369
25%,17.000000,1.0,0.122606,0.122606,0.122606
50%,19.000000,1.0,0.144541,0.144541,0.144541
75%,21.000000,1.0,0.144541,0.144541,0.144541
max,26.000000,1.0,0.144541,0.144541,0.144541


In [90]:
df['cdp'].unique()

array(['4980bf1f-b50b-432a-ab35-8949b00aa37d'], dtype=object)

In [88]:
inv_opts = {
    'width': opts['width']//3,
    'height': opts['height'],
}
top = 100000
top = min(top, len(j_rs))
_df = df.sort_values('ip').iloc[0:top].sort_values('path_len')
p1 = _df.hvplot.box(y='ip', by='path_len', ylim=(0, 1.01*max(df['ip'].max(), 0.1)), title="Distribution of top {} cumulative detection probabilities by path length".format(top), **opts)
p1 = p1+_df.hvplot.hist(y='ip', bins=24, normed=True, invert=True, xlim=(0, 1.01*max(df['ip'].max(), 0.1)), **inv_opts)

top = top // 10
_df = df.sort_values('ip').iloc[0:top].sort_values('path_len')
p2 = _df.hvplot.box(y='ip', by='path_len', ylim=(0, 1.01*max(df['ip'].max(), 0.1)), title="Distribution of top {} cumulative detection probabilities by path length".format(top), **opts)
p2 = p2+_df.hvplot.hist(y='ip', bins=12, normed=True, invert=True, xlim=(0, 1.01*max(df['ip'].max(), 0.1)), **inv_opts)


top = top // 10
_df = df.sort_values('ip').iloc[0:top].sort_values('path_len')
p3 = _df.hvplot.box(y='ip', by='path_len', ylim=(0, 1.01*max(df['ip'].max(), 0.1)), title="Distribution of top {} cumulative detection probabilities by path length".format(top), **opts)
p3 = p3+_df.hvplot.hist(y='ip', bins=6, normed=True, invert=True, xlim=(0, 1.01*max(df['ip'].max(), 0.1)), **inv_opts)

(p1 + p2 + p3).cols(2)

:Layout
   .BoxWhisker.I   :BoxWhisker   [path_len]   (ip)
   .Histogram.I    :Histogram   [ip]   (ip_frequency)
   .BoxWhisker.II  :BoxWhisker   [path_len]   (ip)
   .Histogram.II   :Histogram   [ip]   (ip_frequency)
   .BoxWhisker.III :BoxWhisker   [path_len]   (ip)
   .Histogram.III  :Histogram   [ip]   (ip_frequency)

In [188]:
items = sorted(rs_map.items(), key=lambda x: (len(x[0]), x[0]))#(len(x[0]), len(x[1])))

In [189]:
for i, (key, rs_val) in enumerate(items):
    print()
    print('** i = {}, key len = {}, key = {}, rs_val len = {}'.format(i, len(key), key, len(rs_val)))
    if key not in py_map:
        print('oops, key in rs but not py?')
        continue
    py_val = py_map[key]
    print('.    py_val len = {}'.format(len(py_val)))
    for j, (sub_key, rs_sub_val) in enumerate(sorted(rs_val.items(), key=lambda x: (len(x[0]), x[0]))):
        num_action_nodes = len([acts for acts in [xs[1] for xs in sub_key] if len(acts) > 0])
        if num_action_nodes != 2:
            continue
        print('   @@ j = {}, sub_key len = {}, sub_key = {}'.format(j, len(sub_key), sub_key))
        if sub_key not in py_val:
            print('aargh, sub_key in rs but no py?')
            continue
        py_sub_val = py_val[sub_key]
        print('       rs_sub_val len = {}, rs_sub_val = {}'.format(len(rs_sub_val), rs_sub_val))
        print('       py_sub_val len = {}, py_sub_val = {}'.format(len(py_sub_val), py_sub_val))
        # # print(len(rs_val - py_val), len(py_val - rs_val))
        # print('*** {} in rs, not py'.format(len(rs_val - py_val)))
        # for i, v in enumerate(sorted(rs_val - py_val)):
        #     print(i)
        #     for j in v:
        #         print(j)
        # print('*** {} in py, not rs'.format(len(py_val - rs_val)))
        # for i, v in enumerate(sorted(py_val - rs_val)):
        #     print(i)
        #     for j in v:
        #         print(j)
        # print('*** {} in both'.format(len(py_val & rs_val)))
        # for i, v in enumerate(sorted(py_val & rs_val)):
        #     print(i)
        #     for j in v:
        #         print(j)
        # break


** i = 0, key len = 4, key = ('25d1c743-9a91-473e-9f3b-f1eb011ab252', '94f79fe8-2c11-4d8b-aace-94ebac19375d', '74cf9803-0286-440c-ad20-ad91dacaff22', 'f9ca32a7-ef58-4c4d-b64a-ac2d8664f3ae'), rs_val len = 1
.    py_val len = 1

** i = 1, key len = 5, key = ('25d1c743-9a91-473e-9f3b-f1eb011ab252', '94f79fe8-2c11-4d8b-aace-94ebac19375d', '196a5f4c-8152-4f31-b686-e01d39b05e2e', '74cf9803-0286-440c-ad20-ad91dacaff22', 'f9ca32a7-ef58-4c4d-b64a-ac2d8664f3ae'), rs_val len = 1
.    py_val len = 1

** i = 2, key len = 7, key = ('25d1c743-9a91-473e-9f3b-f1eb011ab252', '1d1f5592-fd01-4347-9445-1b15f70a5517', 'eb827a91-123a-45bd-bc75-b752341922db', '25d1c743-9a91-473e-9f3b-f1eb011ab252', '94f79fe8-2c11-4d8b-aace-94ebac19375d', '74cf9803-0286-440c-ad20-ad91dacaff22', 'f9ca32a7-ef58-4c4d-b64a-ac2d8664f3ae'), rs_val len = 3
.    py_val len = 3

** i = 3, key len = 7, key = ('25d1c743-9a91-473e-9f3b-f1eb011ab252', '94f79fe8-2c11-4d8b-aace-94ebac19375d', '54862ce1-61e3-47c1-ae15-10732a6c16af', 'eb827a9

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

